In [2]:
import sys
import csv
import pandas as pd
from collections import defaultdict
import logging
logging.basicConfig(level=logging.DEBUG)

simp2trad_official='../raw_data_process_cna_cmn/simp2multitrad_official.txt'
if sys.argv[0]=='/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py':
    result='./opencc/ldc_simp2trad_test_opencc_s2tw'
    gold='./test_cases/ldc_simp2trad_gold.csv'
    model='we'
else:
#     logging.basicConfig(filename='eval.log',level=logging.DEBUG)

    
    if len(sys.argv)<4:
        #result='./opencc/sinica_test_opencc_s2tw'
        #gold='./test_cases/sinica_gold.csv'
        logging.warning('python [result file] [gold.csv] [model(we or opencc)]')
        raise
    else:
        result=sys.argv[1]
        gold=sys.argv[2]
        model=sys.argv[3]

logging.info ('result {0} and gold {1},evaluting {2}'.format( result,gold,model))



INFO:root:result ./opencc/ldc_simp2trad_test_opencc_s2tw and gold ./test_cases/ldc_simp2trad_gold.csv,evaluting we


In [3]:
gold_csv = pd.read_csv(gold)
gold_dict=gold_csv.to_dict('records')


In [5]:
multi_trad_official=[]
with open(simp2trad_official) as f:
    for line in f:
        line=line.strip()
        line=line.split('\t')
        simp_char=line[0]
        trad_chars=list(line[2].replace('～',line[0]))
        multi_trad_official+=trad_chars
multi_trad_official


['卜',
 '蔔',
 '几',
 '幾',
 '了',
 '瞭',
 '干',
 '乾',
 '幹',
 '才',
 '纔',
 '万',
 '萬',
 '千',
 '韆',
 '丰',
 '豐',
 '云',
 '雲',
 '歷',
 '曆',
 '仆',
 '僕',
 '斗',
 '鬥',
 '丑',
 '醜',
 '朮',
 '術',
 '叶',
 '葉',
 '只',
 '隻',
 '冬',
 '鼕',
 '飢',
 '饑',
 '匯',
 '彙',
 '出',
 '齣',
 '發',
 '髮',
 '台',
 '臺',
 '颱',
 '檯',
 '朴',
 '樸',
 '夸',
 '誇',
 '划',
 '劃',
 '當',
 '噹',
 '吁',
 '籲',
 '曲',
 '麴',
 '團',
 '糰',
 '回',
 '迴',
 '朱',
 '硃',
 '伙',
 '夥',
 '向',
 '嚮',
 '后',
 '後',
 '合',
 '閤',
 '沖',
 '衝',
 '盡',
 '儘',
 '纖',
 '縴',
 '壇',
 '罈',
 '垻',
 '壩',
 '折',
 '摺',
 '克',
 '剋',
 '蘇',
 '囌',
 '鹵',
 '滷',
 '里',
 '裏',
 '困',
 '睏',
 '別',
 '彆',
 '余',
 '餘',
 '谷',
 '穀',
 '系',
 '係',
 '繫',
 '沈',
 '瀋',
 '表',
 '錶',
 '范',
 '範',
 '板',
 '闆',
 '松',
 '鬆',
 '郁',
 '鬱',
 '制',
 '製',
 '刮',
 '颳',
 '征',
 '徵',
 '舍',
 '捨',
 '卷',
 '捲',
 '帘',
 '簾',
 '彌',
 '瀰',
 '胡',
 '鬍',
 '咸',
 '鹹',
 '面',
 '麵',
 '鍾',
 '鐘',
 '种',
 '種',
 '秋',
 '鞦',
 '復',
 '複',
 '須',
 '鬚',
 '姜',
 '薑',
 '獲',
 '穫',
 '惡',
 '噁',
 '致',
 '緻',
 '党',
 '黨',
 '借',
 '藉',
 '臟',
 '髒',
 '准',
 '準',
 '症',
 '癥',
 '涂',
 '塗',
 '家'

In [12]:
yitizi={'裡':'裏','衚':'胡','覆':'復','畫':'劃'}


In [4]:
a=[1,2,3]
b=[2,3,4]
c=list(zip(a,b))
c

[(1, 2), (2, 3), (3, 4)]

In [6]:

trad2simp={}
line_num=0
num_incor=0
error_per_char=defaultdict(int)
num_per_char=defaultdict(int)

with open(result+'_error', 'w') as error_f:
# #    fieldnames_score=['char_gold',"char_orig",'error_rate']
#     writer_score=csv.DictWriter(score_f,fieldnames=fieldnames_score)
#     writer_score.writeheader()
    with open(result, 'r') as res_f:
        fieldnames = ['char_res','orig_char', 'gold_char','char_index','res','orig','gold','orig_line_num','line_num_in_gold']
        writer = csv.DictWriter(error_f, fieldnames=fieldnames)

        writer.writeheader()
        for line in res_f:
            line=line.rstrip()
            char_is=[i for i in gold_csv['char_index'][line_num].split('-') if i!='']
            char_golds=[i for i in gold_csv['gold_char'][line_num].split('-') if i!='']
            for char_i,char_gold in list(zip(char_is,char_golds)):
                if model=='opencc':
                    char_res=line[char_i]
                elif model=='we':
                    char_res=line[0]
                char_orig=gold_csv['orig_char'][line_num]
                trad2simp[char_gold]=char_orig
                num_per_char[char_gold]+=1

                if char_gold in yitizi:
                    char_gold=yitizi[char_gold]
                if char_res in yitizi:
                    char_res=yitizi[char_res]

                if char_gold!=char_res:
                    if char_gold not in multi_trad_official:
                        logging.warning('gold {1} for {0} not in multi_trad_official. Might be yitizi'.format(char_res,char_gold))

    #                 
    #                     if char_gold in yitizi:
    #                     if yitizi[char_gold]==char_res:

    #                         error_per_char[char_gold]=error_per_char[char_gold]

    #                 if char_res in yitizi:

    #                     if yitizi[char_res]==char_gold:

    #                         error_per_char[char_gold]=error_per_char[char_gold]


                    if char_res not in multi_trad_official:
                            logging.warning('{0} for {1} not in multi_trad_official. Might be yitizi'.format(char_res,char_gold))
                    gold_dict[line_num]['char_res']=char_res
                    gold_dict[line_num]['res']=line
                    gold_dict[line_num]['line_num_in_gold']=line_num
                    writer.writerow(gold_dict[line_num])
                    num_incor+=1
                    error_per_char[char_gold]+=1
                else:
                    error_per_char[char_gold]=error_per_char[char_gold]
                line_num+=1 
            
with open(result+'_score','w') as score_f:
    fieldnames_score=['char_gold',"char_orig",'error_num','total']
    writer_score=csv.DictWriter(score_f,fieldnames=fieldnames_score)
    writer_score.writeheader()
    for key in error_per_char:
        writer_score.writerow ({'char_gold':key, 'char_orig': trad2simp[key],'error_num':int(error_per_char[key]), 'total':num_per_char[key]})
    writer_score.writerow ({'char_gold':'total', 'char_orig': 'total','error_num':num_incor, 'total':len(gold_dict)})

logging.info ('{0} incorrect cases'.format(num_incor))
logging.info('of total {0}'.format(len(gold_csv)))
logging.info ('error rate {0}'.format(float(num_incor/len(gold_dict))))

12264 incorrect cases
of total 165829
error rate 0.07395570135501028
